In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

df = pd.read_csv('clean_data1.csv')
df = df.drop(['Resume'],axis=1)
df.rename(columns={'newer_res':'Resume'},inplace=True)
df.head()


,Unnamed: 0,ID,Category,Resume
0,0,1,HR,"john h. smith , p.h.r . 800-991-5187 | po box ..."
1,1,2,HR,name surname address mobile no/email personal ...
2,2,3,HR,anthony brown hr assistant areas of expertise ...
3,3,4,HR,www.downloadmela.com satheesh email id : caree...
4,4,5,HR,human resources director expert organizational...


In [3]:
from io import StringIO
col = ['Category', 'Resume']
df = df[col]
df = df[pd.notnull(df['Resume'])]
df.columns = ['Category', 'Resume']
df['category_id'] = df['Category'].factorize()[0]
category_id_df = df[['Category', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Category']].values)

In [4]:
res_train, res_test, cat_train, cat_test = train_test_split(df['Resume'], df['Category'], test_size=0.3, random_state = 15)

vectorizer= CountVectorizer()
res_counts= vectorizer.fit_transform(res_train)

tfidf= TfidfTransformer()
res_tfidf= tfidf.fit_transform(res_counts)


classifier=MultinomialNB().fit(res_tfidf, cat_train)


In [5]:
predicted = []
for i in res_test:
    predicted.append((classifier.predict(vectorizer.transform([i])))[0])


In [6]:
a= pd.DataFrame(cat_test)
a['predicted'] = predicted
a.head()


,Category,predicted
94,Managment,Managment
972,Engineering,Engineering
125,Managment,Managment
1064,Building & Construction,Engineering
993,Accountant,Accountant


In [7]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
accuracy=accuracy_score(a.Category, a.predicted)
print("Accuracy from Naive bayes:",accuracy)   

Accuracy from Naive bayes: 0.4444444444444444


In [9]:
from sklearn import metrics

print(metrics.classification_report(a.Category, a.predicted, target_names=df['Category'].unique()))

                         precision    recall  f1-score   support

                     HR       0.60      0.71      0.65        17
              Designing       1.00      0.88      0.94        17
              Managment       1.00      0.11      0.20         9
 Information Technology       0.00      0.00      0.00         5
              Education       0.00      0.00      0.00         2
               Advocate       0.00      0.00      0.00        16
   Business Development       0.00      0.00      0.00         9
       Health & Fitness       0.00      0.00      0.00         6
           Agricultural       0.00      0.00      0.00         5
                    BPO       1.00      0.21      0.35        19
                  Sales       0.00      0.00      0.00         7
             Consultant       0.00      0.00      0.00        16
          Digital Media       0.00      0.00      0.00         9
             Automobile       0.00      0.00      0.00        20
       Food & Beverages 

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
